# Contrastive Learning From Scratch - DistilBERT

An attempt to build contrastive learning model from scratch. Parts include:

- Loading and preparing Wiki-1M data for model input
- Contrastive learning model
  - Forward passing using pre-trained model
  - Constrastive layer
  - Calculate loss
- Training procedure
  - Default trainer optimizer
  - Default trainer hyper-parameters

In [1]:
import os

# Set Project home
PROJECT_HOME = os.path.join('/',
                            'Users',
                            'ng-ka',
                            'OMSCS',
                            'DL',
                            'DLProject',
                            'contrastive-learning-in-distilled-models')
%cd {PROJECT_HOME}

# Load project code
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, './src')

#import distilface
import src.distilface as distilface

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models


## 1. Loading and Preparing Wiki-1M data

Use huggingface `datasets` library to load local file data.

In [2]:
import numpy as np

from datasets import load_dataset

data_files = {'train': 'data/training/wiki1m_for_simcse.txt'}
# data_files = {'train': 'data/training/wiki5k.txt'}
datasets = load_dataset('text', data_files=data_files)

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-84caea1147087fa9
Reusing dataset text (C:\Users\ng-ka\.cache\huggingface\datasets\text\default-84caea1147087fa9\0.0.0\4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.00it/s]


In [3]:
# Unsupervised / Self-supervised dataset

column_names = datasets["train"].column_names
sent0_cname = column_names[0]
sent1_cname = column_names[0]

print('column_names:', column_names)
print('sent0_cname:', sent0_cname, '| sent1_cname:', sent1_cname)

column_names: ['text']
sent0_cname: text | sent1_cname: text


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [13]:
def prepare_features(examples):
    total = len(examples[sent0_cname])

    # Avoid "None" fields 
    for idx in range(total):
        if examples[sent0_cname][idx] is None:
            examples[sent0_cname][idx] = " "
        if examples[sent1_cname][idx] is None:
            examples[sent1_cname][idx] = " "
    
    sentences = examples[sent0_cname] + examples[sent1_cname]

    sent_features = tokenizer(
        sentences,
        max_length=32,
        truncation=True,
        padding=True,
    )

    features = {}
    for key in sent_features:
        features[key] = [[sent_features[key][i], sent_features[key][i+total]] for i in range(total)]

    return features

In [14]:
train_dataset = datasets["train"].map(prepare_features,
                                      batched=True,
                                    #   num_proc=24,
                                      remove_columns=column_names)

Loading cached processed dataset at C:\Users\ng-ka\.cache\huggingface\datasets\text\default-84caea1147087fa9\0.0.0\4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8\cache-1293cb42b0393e2c.arrow


In [7]:
train_dataset.num_rows

1000000

In [8]:
str(train_dataset['input_ids'][0][0])

'[101, 26866, 1999, 2148, 2660, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [9]:
str(train_dataset['input_ids'][0][1])

'[101, 26866, 1999, 2148, 2660, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [10]:
train_dataset.features.keys()

dict_keys(['attention_mask', 'input_ids'])

Sentence 1 and Sentence 2 are the same sentence

## 2. Contrastive Learning Model

In [10]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, DistilBertModel, DistilBertPreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling

from src.distilface.modules.pooler import Pooler
from src.distilface.modules.similarity import Similarity

class DistilBertCLModel(DistilBertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_first_last', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.distilbert = DistilBertModel(config)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None):
        if self.training:
            return self.cl_forward(self.distilbert, input_ids, attention_mask)
        else:
            return self.sent_emb(self.distilbert, input_ids, attention_mask)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None):
        batch_size = input_ids.size(0)#64#input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )


pretrained_model_name = 'distilbert-base-uncased'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(pretrained_model_name)

#model = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

#model.eval();


### 2.1 Initial DistilBERT embeddings performance

In [7]:
import senteval
#import SentEval.senteval as senteval
#import SentEval_simcse.senteval as senteval
#import SentEval_simcse.senteval.engine as se_engine


def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [" ".join(s) for s in batch]
    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors="pt",
        padding=True,
    )

    for k in batch:
        batch[k] = batch[k].to(device)

    with torch.no_grad():
        outputs = model3(**batch)

    pooled_result = outputs.pooler_output.cpu()

    return pooled_result


def evaluate_model():
    PATH_TO_DATA = "./data"

    params = {"task_path": PATH_TO_DATA, "usepytorch": True, "kfold": 10}
    tasks = ["STSBenchmark", 'STS12', 'STS13', 'STS14', 'STS15']

    se = senteval.engine.SE(params, batcher, prepare)
    #se = se_engine.SE(params, batcher, prepare)
    results = se.eval(tasks)

    print('STS12: ', results["STS12"]["all"]["spearman"]["all"])
    print('STS13: ', results["STS13"]["all"]["spearman"]["all"])
    print('STS14: ', results["STS14"]["all"]["spearman"]["all"])
    print('STS15: ', results["STS15"]["all"]["spearman"]["all"])
    print('STSB: ', results["STSBenchmark"]["test"]["spearman"][0])

    return results

In [17]:
results = evaluate_model()
results

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


STS12:  0.47596608898755144
STS13:  0.618674507613396
STS14:  0.5294342415078909
STS15:  0.6969415778069489
STSB:  0.5905314299135291


{'STSBenchmark': {'train': {'pearson': (0.607072923019217, 0.0),
   'spearman': SpearmanrResult(correlation=0.5955188195020807, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.6714570536745258, 3.2552287443463876e-197),
   'spearman': SpearmanrResult(correlation=0.6842681833563105, pvalue=1.122076247507796e-207),
   'nsamples': 1500},
  'test': {'pearson': (0.5632499403360769, 2.9511246107835275e-116),
   'spearman': SpearmanrResult(correlation=0.5905314299135291, pvalue=2.169060496705334e-130),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.6130684398138361,
    'mean': 0.6139266390099399,
    'wmean': 0.6112621097209918},
   'spearman': {'all': 0.6145998749710233,
    'mean': 0.6234394775906401,
    'wmean': 0.6101509979372606}}},
 'STS12': {'MSRpar': {'pearson': (0.42118590394249555, 1.3145639639932755e-33),
   'spearman': SpearmanrResult(correlation=0.45991561737043646, pvalue=1.5821923016995031e-40),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.6352879593428346

## 3. Trainer

In [ ]:
import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator

training_args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=5e-05,
    per_device_train_batch_size= 256,
    per_device_eval_batch_size = 256,
    weight_decay=0.0,
    num_train_epochs=2,
    max_steps= 30000,
    logging_steps=5000,
    save_steps=5000
)

model.train()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

In [21]:
os.getcwd()

'C:\\Users\\ng-ka\\OMSCS\\DL\\DLProject\\contrastive-learning-in-distilled-models'

In [11]:
model_path = 'trained_model/distilbert_cl'

#train_result = trainer.train(model_path=model_path)
train_result = trainer.train()
torch.save(model, './batch64_model.pth')

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000000
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
10000,0.000100
20000,0.000100
30000,0.000100


Saving model checkpoint to output\checkpoint-10000
Configuration saved in output\checkpoint-10000\config.json
Model weights saved in output\checkpoint-10000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-10000\tokenizer_config.json
Special tokens file saved in output\checkpoint-10000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-20000
Configuration saved in output\checkpoint-20000\config.json
Model weights saved in output\checkpoint-20000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-20000\tokenizer_config.json
Special tokens file saved in output\checkpoint-20000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-30000
Configuration saved in output\checkpoint-30000\config.json
Model weights saved in output\checkpoint-30000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-30000\tokenizer_config.json
Special tokens file saved in output\checkpoint-30000\special_tokens_map.json


Training co

In [13]:
mlflow.end_run()

## 4. Evaluate DistilBert CL Model performance

In [12]:
model.eval()

results = evaluate_model()
results

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


STS12:  0.6090776121491925
STS13:  0.7497469551546061
STS14:  0.6757974078475796
STS15:  0.765551098261831
STSB:  0.7204600322058549


{'STSBenchmark': {'train': {'pearson': (0.7471983353662978, 0.0),
   'spearman': SpearmanrResult(correlation=0.7262310424718174, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.7549626323284834, 8.464706174718347e-277),
   'spearman': SpearmanrResult(correlation=0.7611009444961765, pvalue=6.419079616838876e-284),
   'nsamples': 1500},
  'test': {'pearson': (0.7278060625526783, 7.928917149564817e-228),
   'spearman': SpearmanrResult(correlation=0.7204600322058549, pvalue=3.9122309068436553e-221),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.7436187470522633,
    'mean': 0.7433223434158198,
    'wmean': 0.7454487411652428},
   'spearman': {'all': 0.7357581756301707,
    'mean': 0.7359306730579496,
    'wmean': 0.7313708929446705}}},
 'STS12': {'MSRpar': {'pearson': (0.6338161318073211, 1.6730762578990187e-85),
   'spearman': SpearmanrResult(correlation=0.6244768196631411, pvalue=2.451473574472343e-82),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.7875987979312923, 1

# 128 batch size

In [14]:
model2 = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)

training_args2 = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=5e-05,
    per_device_train_batch_size= 128,
    per_device_eval_batch_size = 128,
    weight_decay=0.0,
    num_train_epochs=2,
    max_steps= 30000,
    logging_steps=10000,
    save_steps=10000
)

model2.train()

trainer2 = Trainer(
    model=model2,
    args=training_args2,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at C:\Users\ng-ka/.cache\huggingface\transformers\9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertCLModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequ

In [15]:
model_path = 'trained_model/distilbert_cl'

#train_result = trainer.train(model_path=model_path)
train_result2 = trainer2.train()
torch.save(model2, './batch128_model.pth')

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000000
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
10000,0.000200
20000,0.000200
30000,0.000100


Saving model checkpoint to output\checkpoint-10000
Configuration saved in output\checkpoint-10000\config.json
Model weights saved in output\checkpoint-10000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-10000\tokenizer_config.json
Special tokens file saved in output\checkpoint-10000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-20000
Configuration saved in output\checkpoint-20000\config.json
Model weights saved in output\checkpoint-20000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-20000\tokenizer_config.json
Special tokens file saved in output\checkpoint-20000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-30000
Configuration saved in output\checkpoint-30000\config.json
Model weights saved in output\checkpoint-30000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-30000\tokenizer_config.json
Special tokens file saved in output\checkpoint-30000\special_tokens_map.json


Training co

In [20]:
model2.eval()

results2 = evaluate_model()
results2

STS12:  0.6065243466186726
STS13:  0.756455395306513
STS14:  0.6951194522633876
STS15:  0.7856655374420445
STSB:  0.7158024372787648


{'STSBenchmark': {'train': {'pearson': (0.7717160236711569, 0.0),
   'spearman': SpearmanrResult(correlation=0.7483613114635259, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.7803909383217282, 8.988180520442471e-308),
   'spearman': SpearmanrResult(correlation=0.7817877772593304, pvalue=1.35640744613233e-309),
   'nsamples': 1500},
  'test': {'pearson': (0.7279218768179069, 6.193484465679416e-228),
   'spearman': SpearmanrResult(correlation=0.7158024372787648, pvalue=5.3166095979870885e-217),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.7647390756619414,
    'mean': 0.7600096129369307,
    'wmean': 0.7662246286161297},
   'spearman': {'all': 0.7523686653759916,
    'mean': 0.7486505086672071,
    'wmean': 0.7489687536509297}}},
 'STS12': {'MSRpar': {'pearson': (0.6642231149913949, 1.3119492325269426e-96),
   'spearman': SpearmanrResult(correlation=0.6501533351583848, pvalue=2.6025783691238555e-91),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.8086377246714406, 1

In [31]:
mlflow.end_run()

# 256 batch size no grad scaling

In [ ]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, DistilBertModel, DistilBertPreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling

from src.distilface.modules.pooler import Pooler
from src.distilface.modules.similarity import Similarity

from torch.cuda.amp import autocast 

#scaler = GradScaler()

class DistilBertCLModel(DistilBertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_first_last', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.distilbert = DistilBertModel(config)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None):
        with autocast():
            if self.training:
                return self.cl_forward(self.distilbert, input_ids, attention_mask)
            else:
                return self.sent_emb(self.distilbert, input_ids, attention_mask)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None):
        batch_size = input_ids.size(0)#64#input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )


pretrained_model_name = 'distilbert-base-uncased'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(pretrained_model_name)

#model = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

#model.eval();


In [10]:
import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator

model3 = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)

training_args3 = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=5e-05,
    per_device_train_batch_size= 256,
    per_device_eval_batch_size = 256,
    weight_decay=0.0,
    num_train_epochs=2,
    max_steps= 30000,
    logging_steps=10000,
    save_steps=10000
)

model3.train()

trainer3 = Trainer(
    model=model3,
    args=training_args3,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertCLModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
max_steps is given, it will override any value given in num_train_epochs


In [11]:
model_path = 'trained_model/distilbert_cl'

#train_result = trainer.train(model_path=model_path)
train_result3 = trainer3.train()
torch.save(model3, './batch256_model.pth')

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000000
  Num Epochs = 8
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
10000,0.000500
20000,0.000400
30000,0.000300


Saving model checkpoint to output\checkpoint-10000
Configuration saved in output\checkpoint-10000\config.json
Model weights saved in output\checkpoint-10000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-10000\tokenizer_config.json
Special tokens file saved in output\checkpoint-10000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-20000
Configuration saved in output\checkpoint-20000\config.json
Model weights saved in output\checkpoint-20000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-20000\tokenizer_config.json
Special tokens file saved in output\checkpoint-20000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-30000
Configuration saved in output\checkpoint-30000\config.json
Model weights saved in output\checkpoint-30000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-30000\tokenizer_config.json
Special tokens file saved in output\checkpoint-30000\special_tokens_map.json


Training co

In [26]:
#mlflow.end_run()

In [12]:
model3.eval()

results3 = evaluate_model()
results3

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


STS12:  0.5697409578579856
STS13:  0.6769343320004665
STS14:  0.6357083664225791
STS15:  0.744575188106335
STSB:  0.6320978112448171


{'STSBenchmark': {'train': {'pearson': (0.7108974767715875, 0.0),
   'spearman': SpearmanrResult(correlation=0.6957934629797473, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.7092411378218643, 1.0445253840454627e-229),
   'spearman': SpearmanrResult(correlation=0.7145470998781457, pvalue=1.0790202728598763e-234),
   'nsamples': 1500},
  'test': {'pearson': (0.6362564654216417, 2.357894834089188e-157),
   'spearman': SpearmanrResult(correlation=0.6320978112448171, pvalue=1.0318637044184543e-154),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.694542147265105,
    'mean': 0.6854650266716978,
    'wmean': 0.6986797596788475},
   'spearman': {'all': 0.692993221621729,
    'mean': 0.68081279136757,
    'wmean': 0.6888734527346301}}},
 'STS12': {'MSRpar': {'pearson': (0.5685692938261935, 1.922991733409069e-65),
   'spearman': SpearmanrResult(correlation=0.5768180019887325, pvalue=9.865649661539039e-68),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.7231459307017963, 2.46

In [17]:
mlflow.end_run()

# 256 batch size with grad scaling

In [11]:
class DistilBertCLModel(DistilBertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_first_last', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.distilbert = DistilBertModel(config)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None):
        if self.training:
            return self.cl_forward(self.distilbert, input_ids, attention_mask)
        else:
            return self.sent_emb(self.distilbert, input_ids, attention_mask)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None):
        batch_size = input_ids.size(0)#64#input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )

In [15]:
import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator

model4 = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)

training_args4 = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=5e-05,
    per_device_train_batch_size= 256,
    per_device_eval_batch_size = 256,
    weight_decay=0.0,
    num_train_epochs=2,
    max_steps= 30000,
    logging_steps=10000,
    save_steps=10000,
    fp16=True
)

model4.train()

trainer4 = Trainer(
    model=model4,
    args=training_args4,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertCLModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


In [16]:
model_path = 'trained_model/distilbert_cl'

#train_result = trainer.train(model_path=model_path)
train_result4 = trainer4.train()
torch.save(model4, './batch256_16bit_model.pth')

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000000
  Num Epochs = 8
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
10000,0.000400
20000,0.000300
30000,0.000300


Saving model checkpoint to output\checkpoint-10000
Configuration saved in output\checkpoint-10000\config.json
Model weights saved in output\checkpoint-10000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-10000\tokenizer_config.json
Special tokens file saved in output\checkpoint-10000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-20000
Configuration saved in output\checkpoint-20000\config.json
Model weights saved in output\checkpoint-20000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-20000\tokenizer_config.json
Special tokens file saved in output\checkpoint-20000\special_tokens_map.json
Saving model checkpoint to output\checkpoint-30000
Configuration saved in output\checkpoint-30000\config.json
Model weights saved in output\checkpoint-30000\pytorch_model.bin
tokenizer config file saved in output\checkpoint-30000\tokenizer_config.json
Special tokens file saved in output\checkpoint-30000\special_tokens_map.json


Training co

In [17]:
#import senteval
#import SentEval.senteval as senteval
#import SentEval_simcse.senteval as senteval
#import SentEval_simcse.senteval.engine as se_engine


def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [" ".join(s) for s in batch]
    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors="pt",
        padding=True,
    )

    for k in batch:
        batch[k] = batch[k].to(device)

    with torch.no_grad():
        outputs = model4(**batch)

    pooled_result = outputs.pooler_output.cpu()

    return pooled_result


def evaluate_model():
    PATH_TO_DATA = "./data"

    params = {"task_path": PATH_TO_DATA, "usepytorch": True, "kfold": 10}
    tasks = ["STSBenchmark", 'STS12', 'STS13', 'STS14', 'STS15']

    se = senteval.engine.SE(params, batcher, prepare)
    #se = se_engine.SE(params, batcher, prepare)
    results = se.eval(tasks)

    print('STS12: ', results["STS12"]["all"]["spearman"]["all"])
    print('STS13: ', results["STS13"]["all"]["spearman"]["all"])
    print('STS14: ', results["STS14"]["all"]["spearman"]["all"])
    print('STS15: ', results["STS15"]["all"]["spearman"]["all"])
    print('STSB: ', results["STSBenchmark"]["test"]["spearman"][0])

    return results

In [18]:
model4.eval()

results4 = evaluate_model()
results4

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models\senteval\sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


STS12:  0.6028650839372993
STS13:  0.740729745204717
STS14:  0.692465335997525
STS15:  0.779843190057093
STSB:  0.7226870749431175


{'STSBenchmark': {'train': {'pearson': (0.7654012355643011, 0.0),
   'spearman': SpearmanrResult(correlation=0.7482016388305586, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.7670311528467801, 5.261345910632041e-291),
   'spearman': SpearmanrResult(correlation=0.7708464868002052, pvalue=1.119990676071557e-295),
   'nsamples': 1500},
  'test': {'pearson': (0.7318742172557201, 1.2511244745112543e-231),
   'spearman': SpearmanrResult(correlation=0.7226870749431175, pvalue=3.856394414212571e-223),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.7581791623069682,
    'mean': 0.7547688685556003,
    'wmean': 0.7603260289899136},
   'spearman': {'all': 0.7518720772834617,
    'mean': 0.7472450668579604,
    'wmean': 0.7480605503226412}}},
 'STS12': {'MSRpar': {'pearson': (0.6638885051937329, 1.767074037152879e-96),
   'spearman': SpearmanrResult(correlation=0.6412098115217912, pvalue=4.352737200439051e-88),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.7927758724538693, 5.